# Otimização usando Python

In [3]:
import pyomo.environ as pyo

Na construção de uma rodovia, empregam-se jazidas de rochas para obtenção de pedra britada.  É conveniente transportar este material de jazidas em pedreiras localizadas nas proximidades para alguns pontos preestabelecidos ao longo do caminho em que passará a estrada.

| Depósitos | Pedreira 1 | Pedreira 2 | Pedreira 3 | Oferta |
|-----------|-------------|-------------|-------------|--------|
| 1         | 30          | 13          | 21          | 433    |
| 2         | 12          | 40          | 26          | 215    |
| 3         | 27          | 15          | 35          | 782    |
| 4         | 37          | 25          | 19          | 300    |
Demanda:    | 697         | 421         | 612


A função objetivo deverá minimizar os custos de transporte das cargas em função da quantidade transportada. Dessa forma, tem-se:

$x_{ij}$ é a quantidade (em toneladas) da pedreira i para o depósito j.

$\text{C }min = 30x_{11}+13x_{12}+21x_{13}+12x_{21}+40x_{22}+26x_{23}+27x_{31}+15x_{32}+35x_{33}+37x_{41}+25x_{42}+19x_{43}$

em que $\text{C }min$ é o custo mínimo de transporte.

1. Restrição com relação a oferta:

$x_{11} + x_{12}+x_{13} \leq 433$

$x_{21}+x_{22}+x_{23} \leq 215$

$x_{31} + x_{32}+x_{33} \leq 782$

$x_{41}+x_{42}+x_{43} \leq 300$


2. Restrição com relação a demanda:

$x_{11} + x_{21} + x_{31} + x_{41}= 697$

$x_{12} + x_{22} + x_{32} + x_{42}= 421$

$x_{13} + x_{23} + x_{33} + x_{43}= 612$


3. As variáveis não podem ser negativas.


In [32]:
custos = [[30, 13, 21], 
          [12, 40, 26],
          [27, 15, 35],
          [37, 25, 19]]

oferta = [433, 215,782,300]

demanda = [697, 421, 612]

m = len(oferta)
n = len(demanda)

In [33]:
# Modelo
modelo = pyo.ConcreteModel()

# Índices:
modelo.I = pyo.RangeSet(m)
modelo.J = pyo.RangeSet(n)

# Parâmetros:
modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: custos[i-1][j-1])
modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: oferta[i-1])
modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: demanda[j-1])

# Variáveis de decisão:
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)

In [34]:
# Função objetivo:
def regra_z(mod):
     return sum(mod.c[i,j]*mod.x[i,j] for i in mod.I for j in mod.J)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.minimize) 

# Restrições:
def regra_oferta(mod, i):
    return sum(mod.x[i,j] for j in mod.J) <= mod.a[i]

modelo.restr_oferta = pyo.Constraint(modelo.I, rule=regra_oferta)

def regra_demanda(mod, j):
    return sum(mod.x[i,j] for i in mod.I)== mod.b[j]

modelo.restr_demanda = pyo.Constraint(modelo.J, rule=regra_demanda)

In [35]:
otimizacao = pyo.SolverFactory('glpk').solve(modelo)

In [36]:
modelo.x.pprint()

x : Size=12, Index=I*J
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 : 121.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 : 312.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 : 215.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 : 482.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 : 300.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (4, 3) :     0 : 300.0 :  None : False : False : NonNegativeReals


Solução otimizada:

| Depósitos | Pedreira 1 | Pedreira 2 | Pedreira 3    | 
|-----------|-------------|-------------|-------------|
| 1         |             | 121         | 312         | 
| 2         | 215         |             |             | 
| 3         | 482         | 300         |             | 
| 4         |             |             | 300         | 



In [37]:
print("Custo mínimo:",modelo.z())

Custo mínimo: 33919.0
